# KNN model creation

In [28]:
import sys
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Input, LSTM, GRU



print("Notebook location:", Path().resolve())
print("Current sys.path:", sys.path)


Notebook location: /Users/hippolyteheger/code/bennystu/Trend-Surgeon/notebooks
Current sys.path: ['/Users/hippolyteheger/code/hippolyteheger/03-Decision-Science', '/Users/hippolyteheger/code/bennystu/Trend-Surgeon/notebooks', '/Users/hippolyteheger/.pyenv/versions/3.10.6/lib/python310.zip', '/Users/hippolyteheger/.pyenv/versions/3.10.6/lib/python3.10', '/Users/hippolyteheger/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload', '', '/Users/hippolyteheger/.pyenv/versions/3.10.6/envs/Trend-Surgeon/lib/python3.10/site-packages', '/Users/hippolyteheger/code/bennystu/Trend-Surgeon/time-series-cross-validator', '/Users/hippolyteheger/code/bennystu/Trend-Surgeon/time-series-cross-validator']


In [2]:

pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [20]:
# Set the correct project root where ts_boilerplate lives
project_root = Path("/Users/hippolyteheger/code/bennystu/Trend-Surgeon/time-series-cross-validator")
sys.path.append(str(project_root))

from ts_boilerplate.generate_dummy_data import generate_time_series


In [21]:
X, y = generate_time_series(n_samples=500, input_length=10, n_features=5)


In [22]:
print(X.shape)
print(y.shape)

(500, 10, 5)
(500, 1)


In [29]:
def build_rnn_model(rnn_type='SimpleRNN', units=4, activation='tanh', optimizer='adam', input_shape=(10,5)):
    model = Sequential()
    model.add(Input(shape=input_shape))

    rnn_layer_map = {
        'SimpleRNN': SimpleRNN,
        'LSTM': LSTM,
        'GRU': GRU
    }

    if rnn_type not in rnn_layer_map:
        raise ValueError("rnn_type must be one of ['SimpleRNN','LSTM','GRU']")

    model.add(rnn_layer_map[rnn_type](units=units, activation=activation))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer=optimizer, loss='mse')
    return model

# -----------------------------
# 3️⃣ Hyperparameter grid
# -----------------------------
rnn_types = ['SimpleRNN', 'LSTM', 'GRU']
units_list = [4, 8, 16]
activations = ['tanh', 'relu']
optimizers = ['adam', 'sgd']

# -----------------------------
# 4️⃣ Manual grid search
# -----------------------------
best_loss = float('inf')
best_config = None
best_model = None

for rnn_type in rnn_types:
    for units in units_list:
        for activation in activations:
            for optimizer in optimizers:
                print(f"Training {rnn_type} | units={units} | activation={activation} | optimizer={optimizer}")
                model = build_rnn_model(rnn_type, units, activation, optimizer, input_shape=(X.shape[1], X.shape[2]))

                history = model.fit(X, y, validation_split=0.2, epochs=20, batch_size=16, verbose=0)
                val_loss = history.history['val_loss'][-1]

                if val_loss < best_loss:
                    best_loss = val_loss
                    best_config = {'rnn_type': rnn_type, 'units': units, 'activation': activation, 'optimizer': optimizer}
                    best_model = model

print("\n✅ Best configuration:", best_config)
print("Best validation loss:", best_loss)

# -----------------------------
# 5️⃣ Make predictions with best model
# -----------------------------
y_pred = best_model.predict(X[:5])
print("First 5 predictions:\n", y_pred)

Training SimpleRNN | units=4 | activation=tanh | optimizer=adam
Training SimpleRNN | units=4 | activation=tanh | optimizer=sgd
Training SimpleRNN | units=4 | activation=relu | optimizer=adam
Training SimpleRNN | units=4 | activation=relu | optimizer=sgd
Training SimpleRNN | units=8 | activation=tanh | optimizer=adam
Training SimpleRNN | units=8 | activation=tanh | optimizer=sgd
Training SimpleRNN | units=8 | activation=relu | optimizer=adam
Training SimpleRNN | units=8 | activation=relu | optimizer=sgd
Training SimpleRNN | units=16 | activation=tanh | optimizer=adam
Training SimpleRNN | units=16 | activation=tanh | optimizer=sgd
Training SimpleRNN | units=16 | activation=relu | optimizer=adam
Training SimpleRNN | units=16 | activation=relu | optimizer=sgd
Training LSTM | units=4 | activation=tanh | optimizer=adam
Training LSTM | units=4 | activation=tanh | optimizer=sgd
Training LSTM | units=4 | activation=relu | optimizer=adam
Training LSTM | units=4 | activation=relu | optimizer=sgd
